# json2mmaction2
version: 1

info:
- Create standard mola JSON

author: nuno costa

In [3]:
import json
import os
import argparse
import time
import copy
import glob
import shutil
from pathlib import Path
import numpy as np
from tqdm import tqdm

In [4]:
def make_folders(path='../out/'):
    # Create folders
    if os.path.exists(path):
        shutil.rmtree(path)  # delete output folder
    os.makedirs(path)  # make new output folder
    return

In [5]:
def extract_file(src,dst,copy=True):
    extracted=True
    try:
        if copy: 
            if not os.path.exists(os.path.dirname(dst)): 
                os.makedirs(os.path.dirname(dst)) #make sure dir exists
            shutil.copyfile(src, dst)  # raises if missing files
        else: #if not copy only extracting filelist from json
            if not os.path.exists(src): raise
    except:
        print("\n>> missing : {}".format(src))
        extracted=False
    return extracted
#extract_file("/home/administrator/server_data_ext/Recordings/EASYRIDE/P19/INCAR/20210422/Session1/C1_P7_P8_1/gt/rgb/1.png","/home/administrator/Desktop/INCAR_20210427_Session_1_C10_P13_P14_1_rgb/img_1.png")

In [6]:
def rename_filepaths_from_dir(path, img_format = ".png"):
    #because of possible missing files or miss placement of total frames
    # RENAME FRAMES FILEPATHS (OPTIONAL: mmaction2 can have errors if the total frame number is not equal to the number on filename or there are jumps in numbering)
    for subdir, dirs, files in tqdm(os.walk(path)):
        print(subdir)
        #print(dirs)
        print(len(files))
        files.sort()
        for i,file in enumerate(files):
            if file.endswith(img_format):
                newfile="img_"+str(i+1).zfill(5)+ img_format
                if file==newfile: continue
                src = os.path.join(subdir,file)
                dst = os.path.join(subdir,"rename_"+newfile) #rename_ is necessary for not substituing previous file
                try:
                    os.rename(src,dst)
                    #print("\n>> rename success: from {} to {}".format(file,newfile))
                except:
                    print("\n>> rename failed: {}".format(src))
                    continue
    #remove rename_
    for subdir, dirs, files in tqdm(os.walk(path)):
        for i,file in enumerate(files):
            if file.endswith(('.png')) and file.startswith(('rename_')) :
                newfile = file.replace('rename_','')
                src = os.path.join(subdir,file)
                dst = os.path.join(subdir,newfile)
                try:
                    os.rename(src,dst)
                except:
                    print("\n>> remove rename failed: {}".format(src))
                    continue               
    return
#path='/home/user/Desktop/INCAR/rawframes'
#rename_filepaths_from_dir(path)

# SINGLE CASE STUDY

In [7]:
datasets_root_dir='/mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/'
json_dir='/mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/JSONS/INCAR/'
outdir='/home/user/Desktop/DATASETS/AR/INCAR/' #don't forget the last
dataset="INCAR"
json_file = json_dir+'mola.json'
img_number=None #STOP CONDITION : None copies all
copy_images=True #if false only filelist is extracted from json
copy_videos=False
level=2 #level: 1- one level - ; 2-two-level - classes split

### PREPARE DATASET

In [8]:
def write_from_annotation(json_file, data, images, videos, categories, videos_ann, 
                          copy_images, copy_videos, outdir_img, outdir_video, 
                          img_number=None, level=2, min_frames=30, img_format = ".png"):
    # WRITE FILES (COPY & GENERATE FILELIST)
    # image lists
    img_l = []
    saved_img_l= []
    imglist = []
    img_counter = 0 # image counter
    # video lists
    video_l = []
    saved_video_l= []
    videolist = []
    # write files 
    method="for" #TODO: parfor method
    start=time.time()
    if method=="for":
        #WRITE IMAGES
        for x in tqdm(data['annotations'], desc='Annotations %s' % json_file):  
            # extract label and category
            catid = '%g' % x['category_id']
            category = categories[catid]['name']
            label = int(x['category_id']) - 1 #NOTE: MMACTION2 categories are zero-padded;mola JSON Annotations start by convention in 1
            # extract image info from x['image_id']
            image_id='%g' % x['image_id']
            if image_id in img_l: continue # continue to next loop if repeated image_id 
            img_l.append(image_id)
            img = images[image_id]
            h, w, imgf = img['height'], img['width'], img['file_name']
            _, img_ext = os.path.splitext(imgf)
            img_fn = Path(imgf).stem
            img_frame = img['frame_index'] 
            img_frame_original=copy.copy(img_frame)
            # extract video info from img['video_id']
            video_id = '%g' % img["video_id"]
            video = videos[video_id]
            videof= video["name"]
            video_fn = Path(videof).stem
            keys_l=[k for k in list(videos_ann.keys()) if k.find('_'+video_id+'_'+catid)>-1]
            video_key=[k for k in keys_l if img_frame>=videos_ann[k]['frame_start'] and img_frame<=videos_ann[k]['frame_end']]
            if not video_key: continue #IMG FRAME DOES NOT BELONG TO THIS ANNOTATION
            if len(video_key)>1: raise AssertionError("Current rule - An image can only be in one annotation label!")
            video_key= video_key[0] #Only one 
            video_ann= videos_ann[video_key]
            video_start_frame=video_ann["frame_start"] # No zero-padding in the frames
            video_end_frame=video_ann["frame_end"] # No zero-padding 
            # extract total label frames
            total_frames = '%g' % video_ann['label_frames']
            if video_ann['label_frames']<30: continue
            # extract bounding box format is [top left x, top left y, width, height] | [x,y,w,h]
            box = np.array(x['bbox'], dtype=np.float64) 
            box[:2] += box[2:] / 2  # xy top-left corner to center
            box[[0, 2]] /= w  # normalize x & w
            box[[1, 3]] /= h  # normalize y & h
            if (box[2] > 0.) and (box[3] > 0.):  # if w > 0 and imgf > 0
                # New fnames
                img_frame_original= img_frame_original # No zero-padding because frame starts in one
                img_frame=str(int(img_frame_original)-int(video_start_frame) +1)# update img_frame, +1 to not have zero-padding
                img_new_fn = "img_"+img_frame.zfill(5) #img_framenumber (imgid with zeros 00001: image_frame.zfill(5) ) #WARNING: Problem is I don't no the maximum of images at this point
                video_new_fn = video_fn+'_'+'%g' % video_ann['id'] #the video annotation id is obligatory
                # write images - 1st because if copy_images fails the rest should not be done
                imgf = imgf.split(".")[0]+img_format
                src = os.path.join(datasets_root_dir, imgf)
                img_ext = img_format
                dst = os.path.join(outdir_img, video_new_fn, img_new_fn + img_ext)
                if level==2: dst = os.path.join(outdir_img, category, video_new_fn, img_new_fn + img_ext)
                ext=extract_file(src,dst,copy=copy_images)
                if not ext: continue #if image missing from dataset when extracting images dont write nothing more
                # img list: 
                imgline = f'{video_new_fn}/{img_new_fn}\n' # f'{video_new_fn}/{img_new_fn} {total_frames} {label}\n'
                if level==2: imgline = f'{category}/{video_new_fn}/{img_new_fn}\n' # f'{category}/{video_new_fn}/{img_new_fn} {total_frames} {label}\n'
                imglist.append(imgline)
                img_counter += 1
                # rawframe annotation file list: json to txt [ frame_directory total_frames label  ]
                vidline = f'{video_new_fn} {total_frames} {label}\n'
                if level==2: vidline = f'{category}/{video_new_fn} {total_frames} {label}\n' 
                videolist.append(vidline)
            # STOP conditions
            if img_number and img_counter >= img_number: 
                print("STOP CONDITION")
                break
        #remove duplicate paths
        imglist=list(dict.fromkeys(imglist)) 
        videolist=list(dict.fromkeys(videolist)) 
    stop = time.time()
    elapsed=stop-start
    print("time elapsed:", elapsed)
    return imglist, videolist, saved_img_l, saved_video_l

In [9]:
def mola2mmaction2(datasets_root_dir=None, json_file='push_mola.json', outdir='out/', copy_images=True, copy_videos=False, img_number=None, level=2):
    
    # MAKE ROOT DIRS
    make_folders(path=outdir)  # output directory
    videodir_path = 'videos_%s/' % Path(json_file).stem  # folder name (train, val, test) remove other info
    imgdir_path = 'rawframes_%s/' % Path(json_file).stem  # folder name (train, val, test) remove other info
    outdir_video = os.path.join (outdir, videodir_path)
    outdir_img = os.path.join (outdir, imgdir_path)
    if copy_videos: make_folders(path=outdir_video)
    if copy_images: make_folders(path=outdir_img)
    # PARSE JSON ANNOTATIONS
    data=None
    with open(json_file) as f:
        data = json.load(f)
    if not data: raise
    # create image dict {id: image}
    images = {'%g' % x['id']: x for x in data['images']}
    # create video dict {id: video}
    videos = {'%g' % x['id']: x for x in data['videos']}
    # create category dict {id: category}
    categories = {'%g' % x['id']: x for x in data['categories']}
    # create video ann dict {video_id_category_id: video_annotations}
    videos_ann = {'%g' % x['id'] +'_'+'%g' % x['video_id'] +'_'+'%g' % x['category_id']: x for x in data['video_annotations']}
    # WRITE FILES (COPY & GENERATE FILELIST)
    method="from_annotation"
    if method=="from_annotation": 
        imglist, videolist, saved_img_l, saved_video_l=write_from_annotation(json_file, data, images, videos, categories, videos_ann, copy_images, copy_videos, outdir_img, outdir_video, img_number=img_number, level=level)
    
    
    
    return imglist, videolist, saved_img_l, saved_video_l
#imglist, videolist, saved_img_l, saved_video_l = mola2mmaction2(datasets_root_dir=datasets_root_dir, json_file=json_file, outdir=outdir, copy_images=copy_images, copy_videos=copy_videos, img_number=img_number, level=level)

### GENERATE FILELIST

In [10]:
# GENERATE FILELISTS
def generate_filelist(videolist, dataset, json_file, dataset_type):
    #save imglist :mola_{train,val}_rawframes.txt
    filename=f'{dataset}_total{Path(json_file).stem}_{dataset_type}.txt'
    with open(outdir + filename, 'w') as f:
        f.writelines(videolist)

In [11]:
def recheck_rawframes_filelist(rdir, totalfilelistname):
    with open(os.path.join(rdir, totalfilelistname)) as file: totalfilelist=file.readlines()
    #print(len(totalfilelist))
    original=copy.copy(totalfilelist)
    framesdir=os.path.join(rdir,'rawframes')
    for subdir, dirs, files in tqdm(os.walk(framesdir)):
        if files and files[0].endswith(('.png')):
            dirname=os.path.split(subdir)[-1]
            indices = [i for i, s in enumerate(totalfilelist) if dirname in s]
            #print(indices)
            for idx in indices:
                line=totalfilelist[idx].split(' ')
                line[-2]= str(len(files)) #update total_frames: f'{category}/{video_new_fn} {total_frames} {label}\n' 
                totalfilelist[idx]= ' '.join(line)
                print(totalfilelist[idx])
    newfilename=os.path.join(rdir, "recheck_"+totalfilelistname)
    with open(newfilename, 'w') as f:
        f.writelines(totalfilelist)  
    comparision=[original[i]==totalfilelist[i] for i,n in enumerate(original)]
    return comparision

In [12]:
#rdir=outdir
#totalfilelistname=filename
#comparision=recheck_rawframes_filelist(rdir, totalfilelistname) 

#display(sum([not c for c in comparision]))

### SPLIT FILELIST

In [13]:
def split_filelist(path):
    filelist=[]
    with open(path) as f:
        filelist=f.readlines()
    split=['train','val','test']
    for s in split:
        ps=[]
        if s=="train": ps=['P'+str(i+1)+'_' for i in range(8)]
        if s=="val": ps=['P'+str(i+1)+'_' for i in range(8,12)]
        if s=="test": ps=['P'+str(i+1)+'_' for i in range(12,16)]
        filelist_split=[l for l in filelist for p in ps if l.find(p)>-1]
        filelist_split=list(dict.fromkeys(filelist_split))
        fn_split=Path(path).stem.split('_')
        fn_split[-2]=s
        fn_split='_'.join(fn_split)
        
        with open(os.path.join(os.path.dirname(path),fn_split+'.txt'), 'w') as f:
            f.writelines(filelist_split)
    return
#split_filelist("/home/user/Desktop/DATASETS/AR/INCAR_PUSH_RGB/recheck3_INCAR_PUSH_RGB_totalrgb_push_rawframes.txt")

### CONFIG MMACTION2
1. Option (RECOMMENDED): create a customconfig.py file 
2. Option: modify other config and train from script (see folder MMACTION2/TESTS/mmaction2_tutorial)

# MULTI CASE STUDY (For loop script for each json Dataset - extract multiple datasets)

In [16]:
def convert_mola_json(dataset="mola", datasets_root_dir=None, json_dir='../mola/annotations/', outdir='out/', 
                      copy_images=True, copy_videos=False, img_number=None, level=2, rename=False):
    # Convert motionLab JSON file into  labels --------------------------------
    jsons = glob.glob(json_dir + '*.json')
    # Import json
    for json_file in sorted(jsons):
        jsname=os.path.basename(json_file).split(".")[0]
        outdir=os.path.join(outdir, jsname)
        print("\n>>>", jsname, outdir)
        imglist, videolist, saved_img_l, saved_video_l = mola2mmaction2(datasets_root_dir=datasets_root_dir, 
                                                                        json_file=json_file, 
                                                                        outdir=outdir, 
                                                                        copy_images=copy_images, 
                                                                        copy_videos=copy_videos, 
                                                                        img_number=img_number,
                                                                        level=level
                                                                       )
        if copy_images: dataset_type='rawframes'
        # RENAME FRAMES FILEPATHS   (OPTIONAL: mmaction2 can have errors if the total frame number is not equal to the number on filename or there are jumps in numbering)
        if dataset_type=='rawframes' and rename: rename_filepaths_from_dir(os.path.join(outdir, 'rawframes'))
        # GENERATE FILELISTS
        generate_filelist(videolist, dataset, json_file, dataset_type)
        # RECHECK and RENAME FILELIST
        time.sleep(5)
        comparision=recheck_rawframes_filelist(outdir, filename)
        display(sum([not c for c in comparision]))
        # SPLIT FILELIST
        time.sleep(5)
        split_filelist(os.path.join(outdir,'recheck_'+filename))
    return


In [ ]:
#WARNING use / in dir path if forlder
datasets_root_dir="/mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/"
json_dir=datasets_root_dir+"JSONS/INCAR/violent_2c_toC20/"
outdir='/home/user/Desktop/NC/DATASETS/mma2/' #don't forget the last
dataset="INCAR_VIOLENT_2C"
img_number=None
copy_images=True
copy_videos=False
level=2
convert_mola_json(dataset=dataset, datasets_root_dir=datasets_root_dir, json_dir=json_dir, outdir=outdir, 
                  img_number=img_number, copy_images=copy_images, copy_videos=copy_videos, level=level)



>>> nvs_violent /home/user/Desktop/NC/DATASETS/mma2/nvs_violent


Annotations /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/JSONS/INCAR/vi


>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210422/Session1/C10_P7_P8_2/nvs/574.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210422/Session1/C10_P7_P8_2/nvs/575.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210422/Session1/C10_P7_P8_2/nvs/576.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210422/Session1/C10_P7_P8_2/nvs/577.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210422/Session1/C10_P7_P8_2/nvs/578.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210422/Session1/C10_P7_P8_2/nvs/579.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210422/Session1/C10_P7_P8_2/nvs/580.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210422/Session1/C10_P7_P8_2/nvs/581.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCA

Annotations /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/JSONS/INCAR/vi


>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210422/Session1/C10_P8_P7_1/nvs/577.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210422/Session1/C10_P8_P7_1/nvs/578.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210422/Session1/C10_P8_P7_1/nvs/579.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210422/Session1/C10_P8_P7_1/nvs/580.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210422/Session1/C10_P8_P7_1/nvs/581.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210422/Session1/C10_P8_P7_1/nvs/582.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210422/Session1/C10_P8_P7_1/nvs/583.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210422/Session1/C10_P8_P7_1/nvs/584.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCA

...

Annotations /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/JSONS/INCAR/vi


>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210430/Session 1/C11_P10_P11_1/nvs/590.png


Annotations /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/JSONS/INCAR/vi


>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210430/Session 1/C11_P10_P11_2/nvs/571.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210430/Session 1/C11_P10_P11_2/nvs/572.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210430/Session 1/C11_P10_P11_2/nvs/573.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210430/Session 1/C11_P10_P11_2/nvs/574.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210430/Session 1/C11_P10_P11_2/nvs/575.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210430/Session 1/C11_P10_P11_2/nvs/576.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210430/Session 1/C11_P10_P11_2/nvs/577.png

>> missing : /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/INCAR/20210430/Session 1/C11_P10_P11_2/nvs/578.png

>> missing : /mnt/Data-Ext/Datasets/Internal Da

Annotations /mnt/Data-Ext/Datasets/Internal Datasets/EASYRIDE/P19/JSONS/INCAR/vi

In [ ]:
jsons = glob.glob(json_dir + '*.json')
json_file=jsons[0]
print(json_file)
with open(json_file) as f:
    data = json.load(f)
print(data['videos'][0])
# create video dict {id: video}
videos = {'%g' % x['id']: x for x in data['videos']}
print(videos['1'])
js=json.load(open(json_file))
print(js['videos'][0])

# OTHER FUNCTIONS

In [ ]:
def replace_from_annotation(outdir_img, json_file, data, images, videos, categories, videos_ann, level):
    img_l=[]
    for x in tqdm(data['annotations'], desc='Annotations %s' % json_file):  
        # extract label and category
        catid = '%g' % x['category_id']
        label = catid
        category = categories[label]['name']
        # extract image info from x['image_id']
        image_id='%g' % x['image_id']
        if image_id in img_l: continue # continue to next loop if repeated image_id 
        img_l.append(image_id)
        img = images[image_id]
        h, w, imgf = img['height'], img['width'], img['file_name']
        _, img_ext = os.path.splitext(imgf)
        img_fn = Path(imgf).stem
        # extract video info from img['video_id']
        video_id = '%g' % img["video_id"]
        video = videos[video_id]
        videof= video["name"]
        video_fn = Path(videof).stem
        video_new_fn = video_fn #"video_"+video_id
        video_ann= videos_ann[video_id+'_'+catid]
        video_start_frame=video_ann["frame_start"]-1 # zero if starts in  1
        # src, dst
        img_old_fn = "img_"+img_fn.zfill(5) #"img_"+image_id
        img_frame=str(int(img_fn)-int(video_start_frame))
        img_new_fn = "img_"+img_frame.zfill(5) #img_framenumber (imgid with zeros 00001: image_frame.zfill(5) ) Problem is I don't no the maximum of images
        dirpath=os.path.join(outdir_img, video_new_fn)
        if level==2: dirpath = os.path.join(outdir_img, category, video_new_fn)
        src = os.path.join(dirpath, img_old_fn + img_ext)
        dst = os.path.join(dirpath, img_new_fn + img_ext)
        if src==dst: continue
        try:
            os.rename(src,dst)
        except:
            print("\n>> rename failed: {}".format(src))
            continue
    return
json_file='/home/administrator/server_data_ext/Recordings/EASYRIDE/P19/INCAR/push_mola.json'
outdir_img='/home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes/'
level=2
with open(json_file) as f:
    data = json.load(f)
# create image dict {id: image}
images = {'%g' % x['id']: x for x in data['images']}
# create video dict {id: video}
videos = {'%g' % x['id']: x for x in data['videos']}
# create category dict {id: category}
categories = {'%g' % x['id']: x for x in data['categories']}
# create video ann dict {video_id_category_id: video_annotations}
videos_ann = {'%g' % x['video_id']+'_'+'%g' % x['category_id']: x for x in data['video_annotations']}


replace_from_annotation(outdir_img, json_file, data, images, videos, categories, videos_ann, level)

In [ ]:
%debug

# FAILED CODE IDEAS

In [ ]:
#WARNING TO hard to implement when you want to write images
#TO ADD IMGID you need to search for imgid on video
def write_from_video_annotation(data, images, videos, categories, copy_images, copy_videos, outdir_img, outdir_video):
    # WRITE FILES (COPY & GENERATE FILELIST)
    # video lists
    video_l = []
    saved_video_l= []
    videolist = []
    # write files 
    method="for" #TODO: parfor method
    start=time.time()
    if method=="for":
        #WRITE IMAGES
        for x in tqdm(data['video_annotations'], desc='Video Annotations %s' % json_file):  
            # extract video info from img['video_id']
            video_id = '%g' % x["video_id"]
            video = videos[video_id]
            videof= video["name"]
            video_sensor=video["sensor"]
            video_fn = Path(videof).stem
            video_new_fn = "video_"+video_id
            # extract label and category
            catid = '%g' % x['category_id']
            label = catid
            category = categories[catid]
            # extract total label frames
            total_frames = '%g' % x['label_frames']
            # extract category
            category = categories[label]['name']
            frame_start = x['frame_start'] #first frame
            frame_end = x['frame_end'] #end frame
            # extract image info from x['image_id']
            imgdir=os.path.dirname(videof)+'/'+video_sensor+'/'
            img_l= sorted( filter( os.path.isfile, glob.glob(imgdir + '*') ) )
            # write images - 1st because if copy_images fails the rest should not be done
            # image lists
            img_l = []
            saved_img_l= []
            imglist = []
            img_counter = 0 # image counter
            for imgd in img_l:
                src = os.path.join(imgd)
                dst = os.path.join(outdir_img, video_new_fn, img_new_fn + img_ext) #TODO: how to add imgid you need to search 
                if level==2: dst = os.path.join(outdir_img, category, video_new_fn, img_new_fn + img_ext)
                ext=extract_file(src,dst,copy=copy_images)
                if not ext: continue #if image missing from dataset when extracting images dont write nothing more
                # rawframe annotation file list: json to txt [ frame_directory total_frames label  ]
                imgline = f'{video_new_fn} {total_frames} {label}\n' # f'{video_new_fn}/{img_new_fn} {total_frames} {label}\n'
                if level==2: imgline = f'{category}/{video_new_fn} {total_frames} {label}\n' # f'{category}/{video_new_fn}/{img_new_fn} {total_frames} {label}\n'
                imglist.append(imgline)
                img_counter += 1
        #remove duplicate paths
        imglist=list(dict.fromkeys(imglist)) 
    stop = time.time()
    elapsed=stop-start
    print("time elapsed:", elapsed)
    return imglist, videolist, saved_img_l, saved_video_l